In [1]:
# default_exp core

# module name here

> API details.

In [2]:
#hide 
!pip install nbdev marko

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

[nb_2_html](https://github.com/BNIA/dataplay/blob/main/notebooks/04_nb_2_html.ipynb)

In [5]:
#export
import re
from IPython.display import clear_output
import marko

In [6]:
#export
from nbdev import export2html
from nbdev.export2html import read_nb, get_metadata, clean_exports
from nbdev.export import split_flags_and_code, check_re_multi

In [7]:
#export
# 1st fn. passes each cell to decision fn.
def convertNb(nb):  return [cleanCell(c) for c in nb['cells']]

# 2nd fn. returns text or passes cell to 'code cell' processor
def cleanCell(cell): return marko.convert(cell.source) if cell.cell_type == 'markdown' else processCode(cell)

# 3rd fn. Calls getFlags, processSource, processOutput
def processCode(cell):
  x = [cell.source]
  flags = getFlags(x[0])
  x[0] = processSource( x[0], flags )
  if ( len(cell.outputs) ):
    for o in cell.outputs: x.append( processOutput(o, flags) )
  return x

In [8]:
#export
# Detect and stripout and handle flags.
def getFlags(source):
  input_aug = ['#collapse_input_open', '#collapse_input', '#collapse_output_open', '#collapse_output',
               '#hide_input', '#hide_output', '#hide ', '%%capture', '%%javascript', '%%html']
  return [ x for x in input_aug if re.search(x, source) ]

# enclose the text within a collapsable html drawer.
def makeDetails( content, open ):
  return "<details "+('open' if open else '')+"> <summary>Click to toggle</summary> "+content+"</details>"

In [9]:
#export
# Strip Flags from text, make details, hide all.
def processSource (source, flags):
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_input_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_input'): source = makeDetails( source, False )
    if (lbl == '#hide '): source = ''
    if (lbl == '#hide_input'): source = ''
    if (lbl == '%%javascript'): source = ''
    if (lbl == '%%html'): source = ''
    if (lbl == '%%capture'): source = ''
  return source

In [10]:
#export
def processOutput(source, flags):
  if(source.output_type=='error'):  return ''
  if(source.output_type=='stream'):
    if(source.name=='stderr'): return ''
    source.data = {'text/html': source.text} # This will have the stream process as text/html.
  print(source.data.keys())
  if ( hasattr(source.data, 'text/html') ): source=source.data['text/html'];
  elif ( hasattr(source.data, 'application/javascript') ): source='<script>'+source.data['application/javascript']+'</script>'
  elif ( hasattr(source.data, 'image/png') ): source= "<img src=\"data:image/png;base64," + source.data['image/png'] + "\" alt='Image Alt Text'>"
  elif ( hasattr(source.data, 'text/plain') ): source = '' if re.search("<Figure", source.data['text/plain']) else source.data['text/plain'];
  # print(source)
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_output_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_output'): source = makeDetails( source, False )
    if (lbl == '#hide_output'): source = ''
    if (lbl == '#hide '): source = ''
  return source
  #output_type == 'stream' ==> text
  #output_type == 'display_data' ==> data{'application/javascript' or 'text/html' or 'execute_result'}

In [26]:
#export
# Must be in directory of ipynb you want to convert to html.
def publish(fname='index'): 
  sys.path.pop()
  sys.path.append('workspaces/3Diot.github.io/docs/')
  %cd ../
  from blog import core 
  %cd notebooks 
  nb = read_nb(fname+'.ipynb')
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  outp = ''.join( [''.join(c) for c in outp] ) 
  resp = ''.join( [''.join(c) for c in outp] )
  template = open("../docs/_templates/"+meta_jekyll['template']+".html", "r").read() 
  template = template.replace('{{tab}}', meta_jekyll['tab']) 
  template = template.replace('{{title}}', meta_jekyll['title']) 
  template = template.replace('{{filename}}', fname+'.ipynb') 
  txt = template.replace('{{content}}', resp ) 
  f = open("../docs/"+meta_jekyll['filename'].lower().replace(' ', '_')+".html", "w")
  f.write(txt)
  f.close()
  return txt
  # return "<p>Hello, Worrrld!</p>"

In [12]:
# publish('index')

/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


In [13]:
# publish('002_Parcels')

/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks
dict_keys(['text/html', 'text/plain'])
dict_keys(['text/plain'])
dict_keys(['image/png', 'text/plain'])
dict_keys(['text/html', 'text/plain'])
dict_keys(['text/html', 'text/plain'])
dict_keys(['text/html', 'text/plain'])
dict_keys(['text/html', 'text/plain'])


In [14]:
# publish('003_Progress')

/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


In [15]:
ls

000_core.ipynb        002_Parcels.ipynb     006_Tutorials.ipynb  cv_map.geojson
001a_core_test.ipynb  003_Progress.ipynb    013_CV_Misc.ipynb    index.ipynb
001b_core_test.ipynb  004_Scarcity.ipynb    Scarcity.ipynb
001c_Example.ipynb    005_Technology.ipynb  _devserver.ipynb


In [16]:
nb = read_nb('index.ipynb')
outp = convertNb(nb)

In [27]:
#export
def runit(fname='index.ipynb'): 
  nb = read_nb(fname)
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  txt = ''.join( [''.join(c) for c in outp] )
  return txt

In [28]:
!pip install flask

Defaulting to user installation because normal site-packages is not writeable


In [29]:
ls

000_core.ipynb        002_Parcels.ipynb     006_Tutorials.ipynb  cv_map.geojson
001a_core_test.ipynb  003_Progress.ipynb    013_CV_Misc.ipynb    index.ipynb
001b_core_test.ipynb  004_Scarcity.ipynb    Scarcity.ipynb
001c_Example.ipynb    005_Technology.ipynb  _devserver.ipynb


In [30]:
sys.path.append('workspaces/3Diot.github.io/blog/')

In [31]:
from flask import Flask
import re
app = Flask(__name__)

@app.route("/")
def homepage():  
  return publish('index') 
    
@app.route("/parcels")
def parcels():  
  return publish('002_Parcels') 

@app.route("/progress")
def progress():  
  return publish('003_Progress') 

@app.route("/scarcity")
def scarcity():  
  return publish('004_Scarcity') 

@app.route("/technology")
def technology():  
  return publish('005_Technology') 

@app.route("/tutorials")
def tutorials():  
  return publish('006_Tutorials') 
  

app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2022 23:53:18] "GET / HTTP/1.0" 200 -


/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


127.0.0.1 - - [22/Jan/2022 00:17:47] "GET / HTTP/1.0" 200 -


/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


...

/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


127.0.0.1 - - [22/Jan/2022 01:23:18] "GET / HTTP/1.1" 200 -


/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


127.0.0.1 - - [22/Jan/2022 01:26:12] "GET / HTTP/1.1" 200 -


/workspaces/CVminiGames.github.io/blog
/workspaces/CVminiGames.github.io/blog/notebooks


In [ ]:
t = runit()

In [ ]:
import IPython
IPython.display.HTML(t)

In [ ]:
ls

000_core.ipynb        001c_Example.ipynb  013_CV_Misc.ipynb  index.ipynb
001a_core_test.ipynb  002_Parcels.ipynb   _devserver.ipynb
001b_core_test.ipynb  003_Scarcity.ipynb  cv_map.geojson


In [22]:
sys.path.append('workspaces/3Diot.github.io/blog/')

In [ ]:
sys.path